# Single-state methods and Genetic Algorithms

- Random Search
- Hill Climbing
- Simulated Annealing
- Genetic Algorithms

Let us start by importing the random module (for ```random.choice```, ```random.choices```, and ```random.randint```) and the math module (for ```math.e```).

In [1]:
import random
import math

We define the **OneMax** function as the number of ones in a list

In [2]:
def onemax(xs):
    '''
    The OneMax function. It returns the number of ones in a list

    Args:
        xs: A list of integer
    '''
    return len([x for x in xs if x == 1])

In [3]:
onemax([1,0,1,1,0])

3

## Random Search

For random search we generate ```n_iter``` times a new random solution that is accepted only if its fitness is equal or better than the best solution found so far

In [4]:
def random_search(fit, random_sample, n_iter = 100):
    '''
    This function generate a random sample at each execution, 
    compute the fitness and save the best one among all. 
    
    Args:
        fit: The function that we use for compute the fitness of a sample
        random_sample: The function that we use for extract the sample
        n_iter: The number of iteration that the function have to go through

    Returns:
        The best sample obtained after n iteration and the fit value it achives
    '''
    
    # We need a basic sample before starting the execution
    best = random_sample() # we generate a random sample
    best_fit = fit(best) # we compute the fitness of the genrated sample

    # the main cycle
    for i in range(0, n_iter): 
        x = random_sample() # we generate a random sample
        x_fit = fit(x) # we compute the fitness of the genrated sample
        if x_fit >= best_fit: # if the fit is higher than the best one we shift the best fit with the new one
            best = x
            best_fit = x_fit
    return best_fit, best # return the best sample

Let us try with OneMax and different values of $k$

In [5]:
random_sample = lambda: random.choices([0,1], k=20) # we select a random value from the list k times
print(random_sample())

[0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0]


In [6]:
random_search(onemax, random_sample)

(15, [1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1])

## Hill Climbing

Before defining Hill Climbing we will define some utility functions that will be employed in defining the neighbourhood of a given solution. In particular, how to move from and to a number and its representation as a list of numbers in binary.

In [7]:
def as_binary_string(k, n):
    '''
    This function convert an integer k in a string of lengh n of binary number.
    If the number of binary digits exceeds those necessary to represent the integer,
    no othe digits will be added.
    
    Args:
        k: The integer to convert in binary
        n: The lenght of the binary string. Not less, not more.

    Returns:
        The string of digits representing the integer value in binary.
    '''
    xs = [0] * n # create a list of n digits
    i = 0
    while k != 0:
        if k % 2 == 1:
            xs[n-i-1] = 1
        k = k // 2
        i += 1
    return xs

def to_binary(xs):
    '''
    This function convert an string of binary value in to the decimal rappresentation.
    
    Args:
        xs: List composed of binary value

    Returns:
        The integer value.
    '''    
    k = 0
    n = len(xs)
    for i, b in enumerate(xs):
        k += 2**(n-i-1) * b
    return k

In [8]:
as_binary_string(45, 8)

[0, 0, 1, 0, 1, 1, 0, 1]

In [9]:
to_binary([0,1,0,1,1])

11

Selecting one Neighbor randomly when we consider a binary string as a number $k$ represented in binary is simply a choice between $k-1$ and $k+1$ represented as binary strings:

In [10]:
# With this function we compute the integer neighbourhood of the binary string by summing +1 or -1 to the decimal value. 
# In this way we have less access to the state space but we exlore all the close neighbor.

def neigh_int(xs):
    '''
    This funcition compute the k+1 and k-1 neighbors of the binary string and choose randomly one of them.
    
    Args:
        xs: List composed of binary value

    Returns:
        One of two neighbors in the form of a string of binary numbers.
    '''
    n = len(xs)
    k = to_binary(xs) # binary -> decimal
    if k == 0: # if is 0, we only have the upper Neighbor
        return as_binary_string(k+1, n)
    elif k == 2**n - 1: # if is the max value (the string have all digits at 1), we only have the lower Neighbor
        return as_binary_string(k-1, n)
    else: # we choose randomly one of the two neighbors
        return random.choice([as_binary_string(k-1, n), as_binary_string(k+1, n)])

In [11]:
neigh_int([0, 1])

[1, 0]

For the neighbourhood induced by the Hamming distance, we can select a position inside the list and "flip" the corresponding bit:

In [12]:
# With this function we compute the binary  neighbourhood of the string by flipping one bit. 
# In this way we have more access to the state space but if we are unlucky we could miss the best solution.


def neigh_hamming(xs):
    '''
    This funcition random flip one of the digits from the string xs.
    
    Args:
        xs: List composed of binary value

    Returns:
        The binary string with one bit flipped.
    '''
    n = len(xs)
    candidates = []
    pos = random.randint(0, n-1) # randomly choose one flip
    return xs[0:pos] + [1 - xs[pos]] + xs[pos+1:n] # we make that flip turn and we fix the whole string if the modified bit was equal to 1

We can now define Hill Climbing similarly to random search _except_ for the fact that the nexxt solution to consider is selected in among the neighbours of the current best solution:

In [13]:
# The concept of hill climbing is to move from one neighbour to another one, until we find one with the best solution. 
# The limitation that comes with this solution is that the algorithm could get stuck in a local optima 
# (the best solution but for a local set of neighbours, not for all samples).
# The way to solve the problem is to execute more than one time with random start and use the best sample among all.
# It is not a real solution to the problem but just a way to broaden the search and hope not to always get stuck in the local optima.

def hill_climbing(fit, neigh, start, n_iter = 100):
    '''
    This function perform a hill climbing search by starting from a given value, 
    compute the fitness and start searching between all the neighbors the sample
    with the best fit. 
    
    Args:
        fit: The function that we use for compute the fitness of a sample
        neigh: The function that we use for generate neighbors
        start: The sample where we start the execution
        n_iter: The number of iteration that the function have to go through

    Returns:
        The best sample obtained after n iteration and the fit value it achives
    '''
    best = start
    best_fit = fit(start)
    for i in range(0, n_iter):
        x = neigh(best)
        x_fit = fit(x)
        if x_fit >= best_fit:
            best = x
            best_fit = x_fit
    return best_fit, best

We can now try Hill Climbing on the OneMax problem with both the "integer" neighbourhood and the one induced by the Hamming distance:

In [14]:
starting_point = random.choices([0,1], k = 10)
print(onemax(starting_point), starting_point)
print(hill_climbing(onemax, neigh_int, starting_point)) 
print(hill_climbing(onemax, neigh_hamming, starting_point));

3 [1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
(4, [1, 0, 0, 0, 1, 0, 0, 0, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


As expected, the sum achive worse ressult than the binary flip. That's because we don't get stuck in local optima and always find a way to increase the fitness.

## Simulated Annealing

We can now define the simulated annealing. In addition to what we have seen in the pseudocode, we also keep track of the best solution found so far, which is the one we return.

In [15]:
def simulated_annealing(fit, neigh, start, schedule, n_iter = 100):
    '''
    This function perform a simulated annealing search by starting from a given value, 
    compute the fitness and start searching between all the neighbors the sample
    with the best fit. But, even if the solution is not the better we accept it with
    certain probability T (temperature) that decade during the execution.
    
    Args:
        fit: The function that we use for compute the fitness of a sample
        neigh: The function that we use for generate neighbors
        start: The sample where we start the execution
        schedule: The function that give the value of T
        n_iter: The number of iteration that the function have to go through

    Returns:
        The best sample obtained after n iteration and the fit value it achives
    '''
    current = start
    current_fit = fit(start)
    best = current
    best_fit = current_fit
    T = schedule(0, n_iter)
    for i in range(0, n_iter):
        x = neigh(best)
        x_fit = fit(x)
        if x_fit >= current_fit:
            current = x
            current_fit = x_fit
            if x_fit >= best_fit:
                best = x
                best_fit = x_fit
        elif random.random() <= math.e**((x_fit - current_fit)/T):
            current = x
            current_fit = x_fit
        T = schedule(i, n_iter)
    return best_fit, best

A possible schedule simply reduces the temperature with the number of iterations:

In [16]:
def schedule(i, n_iter):
    '''
    Decresea linearly the temperature based on the number of total iterations divided by the i-th iteration we are in
    
    Args:
        i: The value of the current iteration
        n_iter: The number of total iterations

    Returns:
        Value of T
    '''
    return n_iter/(i+1)

We can now check if something improves for the OneMax problem:

In [17]:
starting_point = random.choices([0,1], k = 40)
print(onemax(starting_point),starting_point)
print(simulated_annealing(onemax, neigh_int, starting_point, schedule))
print(simulated_annealing(onemax, neigh_hamming, starting_point, schedule))

21 [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0]
(23, [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1])
(40, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


This time, we even expect the neighbour integer to perform worst than the starting value. That's because it get shaked by some probability but eventually stuck in a local optimum. 
The hamming distance achive better result, in some case even achiving the best value. It performs well. 
Increasing the number of iteration allow the algorithm to achive highness quality of sample and find the optimum global solution.

In [18]:
print(simulated_annealing(onemax, neigh_int, starting_point, schedule, n_iter=1000))
print(simulated_annealing(onemax, neigh_hamming, starting_point, schedule, n_iter=1000))

(23, [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1])
(40, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


## Genetic Algorithms

We start by defining the tournament selection. Notice that ```max``` on a python list of tuples return the maximum one in lexicographic order:

In [19]:
def tournament_selection(pop, fit, k):
    '''
    Given a certain population and a fitness function we select k individuals
    
    Args:
        pop: List of individuals
        fit: Function that evaluate the individuals
        k: Number of winners

    Returns:
        Return the best individual among the k choosen
    '''    
    tournament = random.choices(pop, k=k)
    selected = max([(fit(x), x) for x in tournament]) # (fitness, individual)
    return selected[1]

We define the one point crossover given two parents:

In [20]:
def one_point_crossover(x, y):
    '''
    The function make a crossover by random select an integer and exchange 
    the information between the two parents after that value is reached.
    
    Args:
        x: First sample
        y: Second sample

    Returns:
        Two "new" sample crossed between 2 parents.
    '''
    n = len(x)
    k = random.randint(0,n-1)
    of1 = x[0:k] + y[k:n]
    of2 = y[0:k] + x[k:n]
    return of1, of2

In [21]:
one_point_crossover([1,1,1,1], [0,0,0,0])

([1, 1, 0, 0], [0, 0, 1, 1])

We can now define the bit-flip mutation with a given probability $p_\text{mut}$:

In [22]:
def bit_flip_mutation(x, p_m):
    '''
    The function apply a mutation to the entire indivual, bit after bit, with certain probability.
    
    Args:
        x: The individual
        p_m: The probability where we apply a mutation
        
    Returns:
        The new mutated sample or not.
    '''   
    def flip(b):
        if random.random() < p_m:
            return 1 - b
        else:
            return b
  
    return [flip(b) for b in x]

In [23]:
bit_flip_mutation([0,1,1,1,0], 0.2)

[1, 1, 0, 1, 0]

One essential step is the initial generation of the population. This can be done by generating uniformly at random ```pop_size``` individuals of $n$ bits:

In [24]:
def init_population(pop_size, n):
    '''
    The cuntion initiliaze the population by randomly extract the samples.
    
    Args:
        pop_size: Number of individuals
        n: Lenght of the binary string

    Returns:
        A list containing the population.
    '''
    return [random.choices([0,1], k = n) for _ in range(0, pop_size)]

We also add a function to return the best indvidual in the population and its fitness:

In [25]:
def get_best(pop, fit):
    '''
    Return the best individual among all samples.
    
    Args:
        pop: List of population
        fit: The function that compute the fitness

    Returns:
        The fittest samples among all the population.
    '''
    return max([(fit(x), x) for x in pop]) # (best_finess, individual)

A single generation is done by performing three steps:
- selection (with tournament selection)
- crossover (with one-point crossover)
- mutation (with bit-flip mutation)

In [26]:
def generation(pop, fit, t_size, p_m):
    '''
    This function menage the population by making a selection, a crossover between 
    all population in group of 2 samples and a bit mutation.
    
    Args:
        pop: List of population
        fit: The fitness function
        t_size: Number of individuals in the torunament
        p_m: Probability of mutation

    Returns:
        A list with the new population after a generation.
    '''
    pop_size = len(pop)
    selected = [tournament_selection(pop, fit, t_size) for _ in range(0,pop_size)]
    offsprings = [one_point_crossover(x, y) for x,y in zip(selected[0::2], selected[1::2])] #(x0, x1), (x2, x3), (x4, x5), ...
    offsprings = [ind for pair in offsprings for ind in pair] # flattern the list
    return list(map(lambda x: bit_flip_mutation(x, p_m), offsprings))

A GA simply perform a generational cycle a predefined number of times:

In [27]:
def GA(pop_size, n, fit, t_size = 4, n_gen = 10):
    '''
    This function is the main cycle of the GA. It inizialize the population, 
    make n generation and select the best sample.
    
    Args:
        pop_size: Size of the population
        n: Lengh of binary string
        fit: The fitness function
        t_size: Number of individuals in the torunament
        n_gen: Number of generation
        
    Returns:
       The best sample.
    '''
    p_m = 1/n
    pop = init_population(pop_size, n)
    for i in range(0, n_gen):
        print(get_best(pop, fit))
        pop = generation(pop, fit, t_size, p_m)
    return get_best(pop, fit)

In [28]:
GA(10, 20, onemax, n_gen=20)

(12, [1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1])
(13, [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1])
(15, [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1])
(15, [1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1])
(15, [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1])
(15, [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1])
(15, [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1])
(16, [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1])
(16, [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1])
(17, [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])
(17, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0])
(17, [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0])
(17, [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0])
(18, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])
(18, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,

(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

### Elitism

Now we can force the population to always contain the best individual found so far

In [29]:
# we want to maximise the value while keeping low the weight

objects_value = [3, 5, 1, 9, 1, 4, 8, 2, 4, 2]
objects_weights = [8, 3, 1, 4, 1, 3, 2, 5, 7, 1]
max_capacity = 17 

def knapsack_fitness(ind):
    '''
    Sume the songle value of the object and compute the fitness of the sample.
    
    Args:
        ind: The sample
        
    Returns:
        The total value if is below the max capacity, otherwise we return a big penalty.
    '''
    tot_value = 0
    tot_weight = 0
    for i, choice in enumerate(ind):
        if choice == 1:
            tot_value += objects_value[i]
            tot_weight += objects_weights[i]
    if tot_weight > max_capacity:
        return -100
    else:
        return tot_value

Now, we use the new fitness with alle the algorithm we have implemented before

In [30]:
random_sample = lambda: random.choices([0,1], k=10)
sol = random_search(knapsack_fitness, random_sample)
print(knapsack_fitness(sol), sol)

0 (26, [0, 1, 0, 1, 0, 0, 1, 0, 1, 0])


In [31]:
starting_point = random.choices([0,1], k = 10)
print(starting_point)
sol1 = hill_climbing(knapsack_fitness, neigh_int, starting_point)
sol2 = hill_climbing(knapsack_fitness, neigh_hamming, starting_point)
print(knapsack_fitness(sol1), sol1)
print(knapsack_fitness(sol2), sol2)

[0, 0, 0, 1, 0, 0, 1, 1, 1, 0]
0 (21, [0, 0, 0, 1, 0, 0, 1, 1, 0, 1])
0 (28, [0, 1, 1, 1, 1, 0, 1, 1, 0, 1])


Why are we not reaching higher value with the hamming distance? Because just flip one bit dosn't necessarly mean a better solution. But instead it could give a worse solution.

In [32]:
starting_point = random.choices([0,1], k = 10)
print(starting_point)
sol1 = simulated_annealing(knapsack_fitness, neigh_int, starting_point, schedule)
sol2 = simulated_annealing(knapsack_fitness, neigh_hamming, starting_point, schedule)
print(knapsack_fitness(sol1), sol1)
print(knapsack_fitness(sol2), sol2)

[0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
0 (16, [0, 0, 0, 1, 1, 0, 0, 0, 1, 1])
0 (21, [0, 0, 1, 1, 1, 1, 0, 0, 1, 1])


With annealing we can achive better result some times but not necessarly true. Sometimes it get stuck in a local maxima.

In [33]:
GA(10, 10, knapsack_fitness, n_gen=20)

(14, [1, 0, 1, 0, 0, 0, 1, 1, 0, 0])
(23, [1, 0, 1, 1, 0, 0, 1, 0, 0, 1])
(28, [0, 1, 0, 1, 0, 0, 1, 0, 1, 1])
(26, [0, 1, 0, 1, 0, 0, 1, 0, 1, 0])
(26, [0, 1, 0, 1, 0, 0, 1, 0, 1, 0])
(21, [0, 1, 0, 0, 0, 1, 1, 0, 1, 0])
(25, [0, 0, 0, 1, 0, 1, 1, 0, 1, 0])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(28, [0, 1, 0, 1, 0, 0, 1, 0, 1, 1])
(28, [0, 1, 0, 1, 0, 0, 1, 0, 1, 1])
(27, [0, 0, 0, 1, 0, 1, 1, 0, 1, 1])
(23, [0, 0, 0, 1, 0, 0, 1, 0, 1, 1])
(28, [0, 1, 0, 1, 0, 1, 1, 0, 0, 1])


(28, [0, 1, 0, 1, 0, 1, 1, 0, 0, 1])

With GA we can achive the more higher value. But we can use elitism. How? The GA remains the same, we only have to change how the generation is compute. If the new best fitness is not better then the courrent best fittness, then we simply make a way to not modify the new sample to make it worse.

In [34]:
from functools import reduce

def generation_elitist(pop, fit, t_size, p_m, elitism=True):
  best_fit, best = reduce(max, [(fit(x), x) for x in pop])
  pop_size = len(pop)
  selected = [tournament_selection(pop, fit, t_size) for _ in range(0,pop_size)]
  offsprings = [one_point_crossover(x, y) for x,y in zip(selected[0::2], selected[1::2])]
  offsprings = [ind for pair in offsprings for ind in pair]
  mutated_offsprings = list(map(lambda x: bit_flip_mutation(x, p_m), offsprings))
  best_fit_new, _ = reduce(max, [(fit(x), x) for x in mutated_offsprings])
  if best_fit_new < best_fit:
    # we remove one individual and replace it with the best
    mutated_offsprings[0] = best 
  return mutated_offsprings

def GA_elitism(pop_size, n, fit, t_size = 4, n_gen = 10, elitism = False):
  p_m = 1/n
  pop = init_population(pop_size, n)
  for i in range(0, n_gen):
    print(get_best(pop, fit))
    pop = generation_elitist(pop, fit, t_size, p_m, elitism)
  return get_best(pop, fit)

In [35]:
GA_elitism(10, 10, onemax, n_gen=20, elitism=True)

(7, [1, 1, 1, 0, 1, 0, 1, 0, 1, 1])
(8, [1, 1, 0, 1, 1, 1, 1, 0, 1, 1])
(8, [1, 1, 0, 1, 1, 1, 1, 0, 1, 1])
(9, [1, 1, 0, 1, 1, 1, 1, 1, 1, 1])
(9, [1, 1, 0, 1, 1, 1, 1, 1, 1, 1])
(9, [1, 1, 1, 1, 0, 1, 1, 1, 1, 1])
(9, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0])
(9, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


(10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

If we try with knapsack we achive the best value among all. Is this the oprimum? We don't know. We take the solution ad the optimum in the short distance in time or we make a very long computation so in a long run we could be more confident in the solution. But is still not garanted.

In [36]:
GA_elitism(10, 10, knapsack_fitness, n_gen=20, elitism=True)

(29, [0, 1, 0, 1, 1, 1, 1, 0, 0, 1])
(29, [0, 1, 1, 1, 0, 1, 1, 0, 0, 1])
(29, [0, 1, 1, 1, 0, 1, 1, 0, 0, 1])
(29, [0, 1, 1, 1, 0, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])
(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])


(30, [0, 1, 1, 1, 1, 1, 1, 0, 0, 1])